<a href="https://www.kaggle.com/code/piyushjain572/binarization?scriptVersionId=199436723" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split,cross_val_score

from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score

from sklearn.compose import ColumnTransformer

from sklearn.preprocessing import Binarizer

In [20]:
df = pd.read_csv('/kaggle/input/modified-titanic-dataset/train.csv',usecols=['Age','Fare','SibSp','Parch','Survived'])
df.head()

,Survived,Age,SibSp,Parch,Fare
0,0,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,1,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,0,35.0,0,0,8.0500


### **Note:** *Handle Missing values before encoding numerical values.*

In [21]:
df.dropna(inplace=True)

In [22]:
df['Family'] = df['SibSp'] + df['Parch']
df.drop(columns=['SibSp','Parch'],inplace=True)
df.shape

(714, 4)

In [23]:
df.isnull().sum()

Survived    0
Age         0
Fare        0
Family      0
dtype: int64

In [24]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(['Survived'],axis=1),
                                                df['Survived'],
                                                test_size=0.2,
                                                random_state=2)
X_train.head()

,Age,Fare,Family
3,35.0,53.1000,1
541,9.0,31.2750,6
679,36.0,512.3292,1
14,14.0,7.8542,0
238,19.0,10.5000,0


### Without binarization

In [10]:
dt = DecisionTreeClassifier()
dt.fit(X_train,y_train)

DecisionTreeClassifier()

In [11]:
y_pred = dt.predict(X_test)
accuracy_score(y_test,y_pred)

0.6643356643356644

In [12]:
cross_val_score(dt,X_train,y_train,cv=10,scoring='accuracy').mean()

0.6479128856624319

### With Binarizer

In [38]:
trf = ColumnTransformer(transformers=[
    ('bin',Binarizer(threshold=1,copy=False),[-1])
],remainder='passthrough')

In [39]:
X_train_trf = trf.fit_transform(X_train)
X_test_trf = trf.transform(X_test)

In [40]:
clf = DecisionTreeClassifier()
clf.fit(X_train_trf,y_train)

y_pred2 = clf.predict(X_test_trf)
accuracy_score(y_test,y_pred2)

0.6573426573426573

In [41]:
cross_val_score(clf,X_train_trf,y_train,cv=10,scoring='accuracy').mean()

0.6251058681185724

In [42]:
output = pd.DataFrame(X_train_trf,columns=['Family_after','Age','Fare'])
output['Family_before'] = X_train['Family'].values
output.sample(15)

,Family_after,Age,Fare,Family_before
461,0.0,41.00,19.5000,1
530,0.0,24.00,7.8958,0
77,0.0,32.00,10.5000,0
337,0.0,22.00,9.3500,0
237,0.0,20.00,7.9250,1
104,0.0,24.00,13.0000,0
82,0.0,16.00,9.5000,0
220,0.0,50.00,106.4250,1
142,1.0,16.00,18.0000,2
199,0.0,21.00,8.4333,0
